In [1]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

In [2]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = r"C:\Users\sidor\Downloads\chromedriver" # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir= r"C:\Users\sidor\Downloads" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36" #Ваш user-agent. Узнать можно тут https://юзерагент.рф, смотреть с браузера Chrome
start_time = time() # время начала выполнения программы

In [3]:
def get_load_time(article_url, user_agent):
    #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [4]:
def write_to_file(output_list, filename, base_dir):
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [5]:
def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [6]:
def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
    
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]

# иногда статья располагается не на внешнем сайте, а на ycombinator, тогда article_url у нее не полный, а добавочный, с параметрами. Например item?id=200933. Для этих случаев будем добавлять урл до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
	# иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"

        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.find(class_="titlelink").string,
            "url": article_url,
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

In [7]:
# инициализируем веб драйвер
browser = webdriver.Chrome(executable_path=driver_path)

# перебираем страницы и собираем нужную информацию
for page_number in range(10):
    print("getting page " + str(page_number) + "...")
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")
    
# завершаем работу драйвера
browser.close()
sleep(1)
browser.quit()
end_time = time()
elapsed_time = end_time - start_time
print("run time: {} seconds".format(elapsed_time))

C:\Users\sidor\AppData\Local\Temp/ipykernel_3520/486006656.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=driver_path)


getting page 0...
'NoneType' object has no attribute 'string'
HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=3.0)
getting page 1...
'NoneType' object has no attribute 'string'
HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=3.0)
getting page 2...
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
getting page 3...
getting page 4...
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
getting page 5...
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
ge

In [8]:
from concurrent.futures import ThreadPoolExecutor, wait

def run_process(page_number, filename):
    browser = webdriver.Chrome(executable_path=driver_path)
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)
        
        browser.quit()
    else:
        print("Error connecting to hacker news")
        browser.quit()
        
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = r"C:\Users\sidor\Downloads\chromedriver" # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir= r"C:\Users\sidor\Downloads" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36" #Ваш user-agent. Узнать можно тут https://юзерагент.рф, смотреть с браузера Chrome
start_time = time() # время начала выполнения программы

futures = []

with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process, number, filename)
        )
wait(futures)
end_time = time()
elapsed_time = end_time - start_time
print("Elapsed run time: {} seconds".format(elapsed_time))

C:\Users\sidor\AppData\Local\Temp/ipykernel_3520/230254206.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=driver_path)


Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x008578B3+2193587]
	Ordinal0 [0x007F0681+1771137]
	Ordinal0 [0x007041A8+803240]
	Ordinal0 [0x007324A0+992416]
	Ordinal0 [0x0073273B+993083]
	Ordinal0 [0x0075F7C2+1177538]
	Ordinal0 [0x0074D7F4+1103860]
	Ordinal0 [0x0075DAE2+1170146]
	Ordinal0 [0x0074D5C6+1103302]
	Ordinal0 [0x007277E0+948192]
	Ordinal0 [0x007286E6+952038]
	GetHandleVerifier [0x00B00CB2+2738370]
	GetHandleVerifier [0x00AF21B8+2678216]
	GetHandleVerifier [0x008E17AA+512954]
	GetHandleVerifier [0x008E0856+509030]
	Ordinal0 [0x007F743B+1799227]
	Ordinal0 [0x007FBB68+1817448]
	Ordinal0 [0x007FBC55+1817685]
	Ordinal0 [0x00805230+1856048]
	BaseThreadInitThunk [0x7532FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77007A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77007A6E+238]
Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x008578B3+2193587]
	Ordinal0 [0x007F0681+1771137]
	Ordinal0 [0x007041A8+803240]
	Ordinal0 [0x007324A0+992416]
	Ordinal0 [0x0073273B+993083]
	Ordinal0 [0x0075F7C2+1